In [1]:
import numpy as np
import random
import json
import tensorflow as tf
from tensorflow import keras

2023-03-05 13:07:46.234117: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

from tensorflow.keras.layers import *
from tensorflow.keras.models import *


In [3]:
import fitness

ModuleNotFoundError: No module named 'fitness'

In [7]:
from test_train_data import x_test, y_test, x_train, y_train

In [8]:
len(x_train), len(y_train), len(x_test), len(y_test)

(10356, 10356, 3452, 3452)

In [9]:
import generate_sub_model as gsm

In [5]:
print("INFO: BASE MODEL")
model = keras.models.load_model('./../models/base_model.h5')

INFO: BASE MODEL


2023-03-05 13:08:11.954307: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 13:08:12.256748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1621] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 503 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:07:00.0, compute capability: 7.0


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0

In [11]:
def getFitness(model, x_train, y_train, x_test, y_test,epochs=2 , verbose=0):
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, verbose=verbose,validation_data=(x_test, y_test))
    _, test_acc = model.evaluate(x_test, y_test , verbose=verbose)
    return test_acc

In [12]:
print("INFO: ORIGINAL ACCURACY " , getFitness(model,x_train, y_train, x_test, y_test, epochs=10 , verbose=1))

Epoch 1/10


2023-03-03 16:59:32.238424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-03-03 16:59:32.623974: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f8418b9f1a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-03 16:59:32.624274: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-03-03 16:59:32.631253: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-03 16:59:32.853651: I tensorflow/compiler/jit/xla_compilation_cache.cc:480] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


324/324 [==============================] - 9s 15ms/step - loss: 0.0517 - accuracy: 0.9818 - val_loss: 0.0927 - val_accuracy: 0.9690
Epoch 2/10
324/324 [==============================] - 4s 11ms/step - loss: 0.0359 - accuracy: 0.9868 - val_loss: 0.0616 - val_accuracy: 0.9780
Epoch 3/10
324/324 [==============================] - 4s 11ms/step - loss: 0.0353 - accuracy: 0.9871 - val_loss: 0.0500 - val_accuracy: 0.9826
Epoch 4/10
324/324 [==============================] - 3s 10ms/step - loss: 0.0268 - accuracy: 0.9914 - val_loss: 0.0475 - val_accuracy: 0.9852
Epoch 5/10
324/324 [==============================] - 3s 11ms/step - loss: 0.0169 - accuracy: 0.9946 - val_loss: 0.0493 - val_accuracy: 0.9858
Epoch 6/10
324/324 [==============================] - 4s 11ms/step - loss: 0.0200 - accuracy: 0.9919 - val_loss: 0.1039 - val_accuracy: 0.9696
Epoch 7/10
324/324 [==============================] - 3s 11ms/step - loss: 0.0209 - accuracy: 0.9924 - val_loss: 0.0700 - val_accuracy: 0.9789
Epoch 8/10

In [13]:
import chromosomes as ch

In [14]:
chromosomes = ch.gen_chromosomes(model, 100)

In [15]:
np.array(chromosomes).shape

(100, 256)

In [20]:

print("INFO: BREEDING CHROMOSOMES")
def breed(chromosomes,x_train, y_train, x_test, y_test):
    new_chromosomes = []
    new_fitness = []
    for i in range(1):
        print("Breeding for {}th time".format(i+1))
        # randomly select 2 chromosomes
        c1 = random.choice(chromosomes)
        c2 = random.choice(chromosomes)
        # randomly select a crossover point
        crossover_point = random.randint(0, 27)
        # create a new chromosome by combining the 2 chromosomes
        new_chromosome_1 = c1[:crossover_point] + c2[crossover_point:]
        new_chromosome_2 = c2[:crossover_point] + c1[crossover_point:]
        # calculate the fitness of old and new chromosomes
        model1 = gsm.genSubModel(c1, model)
        model2 = gsm.genSubModel(c2, model)
        model3 = gsm.genSubModel(new_chromosome_1, model)
        model4 = gsm.genSubModel(new_chromosome_2, model)
        fitness1 = fitness.getFitness(model1, x_train, y_train, x_test, y_test)
        if(fitness1 < 0.8):
            print("Breaking because of less acuracy")
            continue
        fitness2 = fitness.getFitness(model2, x_train, y_train, x_test, y_test)
        if(fitness2 < 0.8):
            print("Breaking because of less acuracy")
            continue
        fitness3 = fitness.getFitness(model3, x_train, y_train, x_test, y_test)
        if(fitness3 < 0.8):
            print("Breaking because of less acuracy")
            continue
        fitness4 = fitness.getFitness(model4, x_train, y_train, x_test, y_test)
        if(fitness4 < 0.8):
            print("Breaking because of less acuracy")
            continue
        # select the best 2 chromosomes
        fitnesses = [fitness1, fitness2, fitness3, fitness4]
        best_chromosomes = [c1, c2, new_chromosome_1, new_chromosome_2]
        best_chromosomes = [x for _, x in sorted(zip(fitnesses, best_chromosomes), reverse=True)]
        new_chromosomes.append(best_chromosomes[0])
        new_chromosomes.append(best_chromosomes[1])
        new_fitness.append(fitnesses[0])
        new_fitness.append(fitnesses[1])

    return new_chromosomes, new_fitness

INFO: BREEDING CHROMOSOMES


In [21]:
import numpy as np

In [22]:
new_chromosomes, new_fitness = breed(chromosomes, x_train, y_train, x_test, y_test)


Breeding for 1th time


In [24]:
len(new_chromosomes)

2

In [25]:
new_fitness

[0.9701622128486633, 0.9632097482681274]